[Open In Colab](https://colab.research.google.com/github/joelbenjohn/pyCUFSM/blob/master/pyCUFSM_load_mat.ipynb)

In [ ]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

In [73]:
from scipy import io
import pycufsm.helpers as helpers
file = io.loadmat('cnolip_Mx.mat')
nodes, elements, lengths, props, constraints, springs = helpers.load_mat(file)

<class 'dict'>


UnicodeDecodeError: 'utf-32-le' codec can't decode bytes in position 0-3: code point not in range(0x110000)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pycufsm.fsm import strip
from pycufsm.preprocess import stress_gen
from pycufsm.preprocess import yieldMP
from pycufsm.preprocess import doubler
import pycufsm.plotters

A, xc, yc, Ix, Iy, Ixy, theta, I1, I2, J, xs, ys, Cw, B1, B2, wn = cutwp_prop2.cutwp_prop2(nodes[:, 1:3], elements[:, 1:4])
# Values here correspond to signature curve basis and orthogonal based upon geometry
gbt_con = {
    'glob': [0],
    'dist': [0],
    'local': [0],
    'other': [0],
    'o_space': 1,
    'couple': 1,
    'orth': 2,
    'norm': 0,
}
# Simply-supported boundary conditions
b_c = 'S-S'

# For signature curve analysis, only a single array of ones makes sense here
m_all = np.ones((len(lengths), 1))  

# Solve for 10 eigenvalues
n_eigs = 10
#Flag = {node, elem, mat, stress, stresspic, coord, constraints, springs, origin, propaxis}
flag = np.array([0, 0, 0, 0, 1, 1, 1, 0, 0, 0])
pycufsm.plotters.crossect(nodes, elements, springs, constraints, flag)
# Set the section properties for this simple section
# Normally, these might be calculated by an external package
sect_props = {
    'cx': xc,
    'cy': yc,
    'x0': xs,
    'y0': ys,
    'phi': theta,
    'A': A,
    'Ixx': Ix,
    'Ixy': Ixy,
    'Iyy': Iy,
    'I11': I1,
    'I22': I2
}
# Yield Strength
fy = fy[0]
unsymmetric = 0 #i.e. do a restrained bending calculation
# Generate the stress points assuming 500 MPa yield and X-axis bending
Fyield = yieldMP(nodes, fy, sect_props, unsymmetric)
nodes_p = stress_gen(nodes=nodes,
                    forces={
                        'P': Fyield['P']*0,
                        'Mxx': Fyield['Mxx']*1,
                        'Myy': Fyield['Myy']*0,
                        'M11': Fyield['M11']*0,
                        'M22': Fyield['M22']*0
                    },
                    sect_props=sect_props,
                    unsymm = unsymmetric,
                    fy =fy
                    )


In [ ]:
# Perform the Finite Strip Method analysis
signature, curve, shapes = strip(props=props,
                                 nodes=nodes_p,
                                 elements=elements,
                                 lengths=lengths,
                                 springs=springs,
                                 constraints=constraints,
                                 gbt_con=gbt_con,
                                 b_c=b_c,
                                 m_all=m_all,
                                 n_eigs=n_eigs,
                                 sect_props=sect_props)
lengths = np.array(lengths)
signature = np.array(signature)
curves = np.array(curve)
node=np.array(nodes)
shapes = np.array(shapes)
element = np.array(elements)

In [ ]:
######POST PROCESSING#########
#Choose mode number
modeindex = 1
length_index = 20
fileddisplay = [1]
######DISPLACED SHAPE####
undef = 1
scalem = 5
m_a = [1]
SurfPos = 1/2
mode = shapes[length_index-1, :, modeindex-1]
####CALL DISPLACED SHAPE
x = pycufsm.plotters.dispshap(undef, node, element, mode, scalem, springs, m_a, b_c ,SurfPos)

In [ ]:
####SIGNATURE CURVE####
###Buckling halfwavelenth plot from a signature analysis
plt.semilogx(lengths, signature, 'k', linewidth= 2)
plt.xlim((1, 1000))
maximum = np.max(signature)
median = np.median(signature)
ubound = np.min([maximum, 3*median])
plt.ylim((0, ubound))
plt.xlabel('half-wavelength (in)')
plt.ylabel('Pcr/Py')
plt.show()